# Variational Quantum Deflation - Ng Chun Seng

Here, I propose a framework to perform Variational Quantum Deflation (VQD) in order to calculate excited states of Acrolein molecule. -- _without using predefined VQD algorithms_

## Procedure

Using PySCF and OpenFermion libraries (running on Linux subsystem), I have 
- Generated (6-qubits truncated) Qubit Hamiltonian from full-spectrum second-quantized Fermionic Hamiltonian via freezing of occupied spin orbitals 

Let's first initialize the dependencies and import the dataset, i.e.: Qubit Hamiltonian in terms of Pauli (string) bases

In [1]:
import json

with open ("acrolein_qubit.json","r") as file:
    data = json.load(file)

print(data)

{'()': '(-187.63041421019528+0j)', "((0, 'Z'),)": '(0.04831322288376241+0j)', "((1, 'Z'),)": '(0.11376566187096011+0j)', "((1, 'Y'), (2, 'Z'), (3, 'Y'))": '(0.015421406434414281+0j)', "((1, 'X'), (2, 'Z'), (3, 'X'))": '(0.015421406434414281+0j)', "((1, 'Y'), (2, 'Z'), (3, 'Z'), (4, 'Z'), (5, 'Y'))": '(0.013708152913771532+0j)', "((1, 'X'), (2, 'Z'), (3, 'Z'), (4, 'Z'), (5, 'X'))": '(0.013708152913771532+0j)', "((2, 'Z'),)": '(0.08811361570154505+0j)', "((2, 'Y'), (3, 'Z'), (4, 'Y'))": '(0.01070441161656259+0j)', "((2, 'X'), (3, 'Z'), (4, 'X'))": '(0.01070441161656259+0j)', "((3, 'Z'),)": '(-0.01739930707931824+0j)', "((3, 'Y'), (4, 'Z'), (5, 'Y'))": '(0.0009306289712739703+0j)', "((3, 'X'), (4, 'Z'), (5, 'X'))": '(0.0009306289712739703+0j)', "((4, 'Z'),)": '(-0.05929637337331601+0j)', "((5, 'Z'),)": '(-0.0958575197646896+0j)', "((0, 'Z'), (1, 'Z'))": '(0.07552358937978539+0j)', "((0, 'Z'), (1, 'Y'), (2, 'Z'), (3, 'Y'))": '(0.024398960970408744+0j)', "((0, 'Z'), (1, 'X'), (2, 'Z'), (3, 

We create the SparePauliOp object to store the Hamiltonian/Observable:

In [ ]:
from qiskit.quantum_info import SparsePauliOp

n_qubits = 6
qubit_hamiltonian_terms = []

def generate_pauli_strings(data):
    keys = []
    pauli_strings=[]
    for key, value in data.items():
        keys.append(key[2:-2].split("), ("))
    for index,operators in enumerate(keys) :
        if index == 0:
            pauli_strings.append("I"*n_qubits)
            continue
        pauli_string = list("I"*n_qubits)
        for pairs in operators:
            pauli_string[int(pairs[0])] = pairs[4]
        pauli_string = "".join(pauli_string)
        pauli_strings.append(pauli_string)
    #print(pauli_strings)

    for index, value in enumerate(data.values()):
        qubit_hamiltonian_terms.append(tuple([pauli_strings[index],value[1:-1]]))

generate_pauli_strings(data)
print(qubit_hamiltonian_terms)

qubit_hamiltonian = SparsePauliOp.from_list(qubit_hamiltonian_terms)

[('IIIIII', '-187.63041421019528+0j'), ('ZIIIII', '0.04831322288376241+0j'), ('IZIIII', '0.11376566187096011+0j'), ('IYZYII', '0.015421406434414281+0j'), ('IXZXII', '0.015421406434414281+0j'), ('IYZZZY', '0.013708152913771532+0j'), ('IXZZZX', '0.013708152913771532+0j'), ('IIZIII', '0.08811361570154505+0j'), ('IIYZYI', '0.01070441161656259+0j'), ('IIXZXI', '0.01070441161656259+0j'), ('IIIZII', '-0.01739930707931824+0j'), ('IIIYZY', '0.0009306289712739703+0j'), ('IIIXZX', '0.0009306289712739703+0j'), ('IIIIZI', '-0.05929637337331601+0j'), ('IIIIIZ', '-0.0958575197646896+0j'), ('ZZIIII', '0.07552358937978539+0j'), ('ZYZYII', '0.024398960970408744+0j'), ('ZXZXII', '0.024398960970408744+0j'), ('ZYZZZY', '0.0037403972559761477+0j'), ('ZXZZZX', '0.0037403972559761477+0j'), ('ZIZIII', '0.07354273898304513+0j'), ('ZIYZYI', '0.022185537833549488+0j'), ('ZIXZXI', '0.022185537833549488+0j'), ('ZIIZII', '0.0936286269165088+0j'), ('ZIIYZY', '-0.02994641370495954+0j'), ('ZIIXZX', '-0.0299464137049595

Next we construct initialize the circuit and use Two-Local ansatz -- _Refer [here](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.TwoLocal) for details._

<!-- Next we construct initial Hartree-Fock (HF) state $|111000\rangle$ and we use Two-Local ansatz -- _Refer [here](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.TwoLocal) for details._ -->

<!-- we use Unitary Coupled-Cluster Singles and Doubles (UCCSD) ansatz. -- _Refer [here](https://docs.pennylane.ai/en/stable/code/api/pennylane.UCCSD.html) for details._  -->

In [ ]:
import qiskit
import numpy as np
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Estimator
from qiskit.circuit.library import TwoLocal
# from qiskit_nature.second_q.circuit.library import UCCSD
# from qiskit_nature.second_q.mappers import JordanWignerMapper

aer = AerSimulator()
estimator = Estimator()

# #UCCSD ansatz
# uccsd_ansatz = UCCSD(
#     num_spatial_orbitals=3,
#     num_particles=(1,2),
#     initial_state=qc,
#     qubit_mapper=JordanWignerMapper(),
# )
# print(uccsd_ansatz)

two_local_ansatz = TwoLocal(6, rotation_blocks=["ry", "rz"], entanglement_blocks='cz', reps=2)  #36 parameters
print(two_local_ansatz)

     »
q_0: »
     »
q_1: »
     »
q_2: »
     »
q_3: »
     »
q_4: »
     »
q_5: »
     »
«     ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                                                                                                                                                                                         ├
«     │                                                                                                                                                                                                                          │
«q_1: ┤1                                                                                                                                                                                                                         ├
«

As per VQD, we would require Qubit Hamiltonian $H$ to classically optimize and minimize the cost function $\mathcal{C}$ defined as:

$$\mathcal{C}(\lambda_k) = \langle \psi(\lambda_k) | H | \psi(\lambda_k) \rangle + \sum_i^{k-1} \beta_i |\langle \psi(\lambda_k)|\psi(\lambda_i)\rangle |^2$$

where we have $H = \sum_j c_j P_j$ in pauli string bases as obtained from previous procedures.



Thus, we define our cost function, initialize the circuit and run our VQD. 

In [ ]:
from scipy.optimize import minimize
from qiskit.quantum_info import Statevector
from qiskit.primitives import Estimator as Es

number_of_calculated_states = 3

params = np.zeros(two_local_ansatz.num_parameters)
beta = 10       #large penalty term for convergence

def cost_function(params,k) :
    ground_cost = estimator.run(circuits=two_local_ansatz, parameter_values=params, observables=qubit_hamiltonian).result().values[0]
    if k == 0 :
        return ground_cost
    current_state = Statevector(two_local_ansatz.assign_parameters(params)).data
    overlap = 0
    for i in range(len(corresponding_statevector)):
        overlap += np.abs(np.vdot(corresponding_statevector[i], current_state)) ** 2
    cost = ground_cost + beta*overlap
    return cost                           

primitive_estimator = Es()

corresponding_statevector = []
corresponding_parameters = []
cost = []
energy = []
for i in range(number_of_calculated_states):
    result = minimize(cost_function,params,args=i,method="COBYLA")   
    cost.append(result.fun)
    corresponding_parameters.append(result.x)
    corresponding_statevector.append(Statevector(two_local_ansatz.assign_parameters(corresponding_parameters[i])).data)
    energy.append(primitive_estimator.run(circuits=two_local_ansatz, parameter_values=corresponding_parameters[i], observables=qubit_hamiltonian).result().values[0])

print("Our VQD Results:")
for i, energy in enumerate(energy):
    print(f"State {i}: {energy.real:.6f}")

Our VQD Results:
State 0: -188.064080
State 1: -187.969554
State 2: -187.649179


## Cross-check

Let's compare results from our framework with the predefined VQD algorithm in Qiskit as well as exact energies of Acrolein obtained from classical diagonalization. 

### Predefined Qiskit VQD

In [ ]:
from qiskit_algorithms.optimizers import COBYLA
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms import VQD
from qiskit_algorithms.state_fidelities import ComputeUncompute
from qiskit_aer.primitives import Sampler

ansatz = TwoLocal(6, rotation_blocks=["ry", "rz"], entanglement_blocks='cz', reps=2)
optimizer = COBYLA()                # very good (unconstrained) optimizer - works really well with Two Local ansatz 

# Set up Predefined VQD
sampler = Sampler()
estimator = Estimator()
fidelity = ComputeUncompute(sampler)
vqd = VQD(estimator=estimator, fidelity=fidelity, ansatz=ansatz, optimizer=optimizer, k=3)  # Ground + 1st excited state + 2nd excited state

result = vqd.compute_eigenvalues(qubit_hamiltonian)

print("Qiskit VQD Results:")
for i, energy in enumerate(result.eigenvalues):
    print(f"State {i}: {energy.real:.6f}")

Qiskit VQD Results:
State 0: -188.163991
State 1: -185.976269
State 2: -178.248917


### Exact Energies from classical diagonalization 

In [ ]:
from scipy.linalg import eigh 

H_dense = qubit_hamiltonian.to_matrix()
eigenvalue, eigenvectors = eigh(H_dense)

print("Exact Results:")
print("Ground state energy:", eigenvalue[0])
print("First excited state:", eigenvalue[1])
print("Second excited state:", eigenvalue[2])


Exact Results:
Ground state energy: -188.32366518930124
First excited state: -188.18885237429478
Second excited state: -188.14305168081395


Note that the classical diagonalization above is performed on truncated Hamiltonian with frozen orbitals. -- _Same goes to the VQDs above_

Thus, these calculated energies would definitely _deviate_ from actual exact energies obtained from spectroscopy techniques, i.e.: via experiments.

## Post-processing and Analysis 

Note that Ground state energy from our VQD is 0.25958518930124796 higher than exact ground state obtained from classical diagonalization. Since the calculation of excited states are dependent on prior determined states. We could account for the error in ground state approximation, to inspect the actual deviation of excited states from exact classical diagonalizations.

### Our Post-processed VQD results:

State 1: -187.969554-0.25958518930124796 = -188.22913918930124

* First excited state deviates $\sim 0.02\%$ from exact first excited state

Repeating this for second excited state: 

State 2: -187.649179-0.25958518930124796-0.04028681500645348 = -187.9490510043077

* Second excited state deviates $\sim 0.1\%$ from exact second excited state

### Our VQD Energy Gap vs Exact Energy Gap:

* 0 -> 1 : (Our VQD) 0.09452600000000189 vs (Exact Gap) 0.13481281500645537
* 1 -> 2 : (Our VQD) 0.3203749999999843 vs (Exact Gap) 0.04580069348082816


## Conclusion 

* Our VQD is good. - It has faster computation speed (though we could augment and respect Qubit-wise Commutativity (QWC) for even faster and efficient quantum simulation), and approximately equal excited state approximations when compared to exact diagonalization results.

- Predefined VQD from Qiskit sometimes fails to give reasonable results for excited states (if we do not have a good ansatz and a synchronized optimizer), on top of slow computation speed. 

- Our VQD procedure/framework achieves similar ground state energy approximations with the predefined VQD, and on average deviates about $\sim 0.1\%$ from exact ground state energy obtained from classical diagonalization. In the meantime, predefined Qiskit VQD has relatively low accuracy in approximating first and second excited states as seen in results above.

- VQD would be useful for Hamiltonian that is hard to classically diagonalize -- allowing us to access orthogonal subspaces in order to search for excited states, otherwise classical diagonalization is more efficient and sufficient for Hamiltonians truncated to small active space (few spatial orbitals)

- Nonetheless, our VQD procedure could be further enhanced and scaled to solve larger Hamiltonians (or rather full-spectrum Hamiltonians), and our VQD is indeed a proof-of-concept.